In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/Nikkei-Visual-Data-Journalism/Heatwave/main/data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-by-pref-daily.csv'

In [3]:
df = pd.read_csv(url, parse_dates=['date'])

東京都の記録

In [4]:
tokyo = df.loc[df.pref=='東京都'].copy()

In [5]:
tokyo.tail()

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol
405401,2023-08-13,2023,東京都,33.9,32.9,True,True,False,False,False,False
405448,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False
405495,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False
405542,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False
405589,2023-08-17,2023,東京都,34.8,34.5,True,True,False,False,False,False


In [6]:
def count_consecutive_true(group):
    is_true = group['over30_capitol']
    group['consecutive_count'] = (is_true.groupby((is_true != is_true.shift()).cumsum()).cumcount() + 1) * is_true
    return group

tokyo = tokyo.groupby('year').apply(count_consecutive_true).reset_index(drop=True)

In [7]:
tokyo

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
0,2000-01-01,2000,東京都,24.1,12.6,False,False,False,False,False,False,0
1,2000-01-02,2000,東京都,24.1,8.8,False,False,False,False,False,False,0
2,2000-01-03,2000,東京都,23.8,16.5,False,False,False,False,False,False,0
3,2000-01-04,2000,東京都,24.1,14.0,False,False,False,False,False,False,0
4,2000-01-05,2000,東京都,24.2,11.6,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8625,2023-08-13,2023,東京都,33.9,32.9,True,True,False,False,False,False,39
8626,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False,40
8627,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False,41
8628,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False,42


In [8]:
tokyo[(tokyo.year<2023)&(tokyo.consecutive_count>=40)]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
1687,2004-08-14,2004,東京都,37.0,34.7,True,True,True,False,False,False,40


In [9]:
tokyo[(tokyo.year==2023)&(tokyo.consecutive_count>=40)]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
8626,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False,40
8627,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False,41
8628,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False,42
8629,2023-08-17,2023,東京都,34.8,34.5,True,True,False,False,False,False,43


In [10]:
tokyo[tokyo.year==2023]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
8401,2023-01-01,2023,東京都,24.4,13.0,False,False,False,False,False,False,0
8402,2023-01-02,2023,東京都,24.4,12.1,False,False,False,False,False,False,0
8403,2023-01-03,2023,東京都,24.4,11.0,False,False,False,False,False,False,0
8404,2023-01-04,2023,東京都,24.7,11.0,False,False,False,False,False,False,0
8405,2023-01-05,2023,東京都,24.8,10.6,False,False,False,False,False,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8625,2023-08-13,2023,東京都,33.9,32.9,True,True,False,False,False,False,39
8626,2023-08-14,2023,東京都,32.9,32.9,True,True,False,False,False,False,40
8627,2023-08-15,2023,東京都,32.8,32.8,True,True,False,False,False,False,41
8628,2023-08-16,2023,東京都,32.2,32.2,True,True,False,False,False,False,42


In [11]:
tokyo[tokyo.maxtemp>=40]

,date,year,pref,maxtemp,maxtemp_capitol,over30,over30_capitol,over35,over35_capitol,over40,over40_capitol,consecutive_count
6778,2018-07-23,2018,東京都,40.8,39.0,True,True,True,True,True,False,16


heatpoints by pref

In [12]:
url = './data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-by-pref-flourish.csv'

In [13]:
hp_f = pd.read_csv(url)

In [14]:
hp_f[hp_f.pref=='滋賀県']

,year,pref,sort_n,over30_capitol,over30_capitol_roy,over30_capitol_ytd,over35_capitol,over35_capitol_roy,over35_capitol_ytd,temp
48,2000,滋賀県,60,72.0,27.0,45.0,NaN,NaN,NaN,真夏日
49,2000,滋賀県,60,NaN,NaN,NaN,4.0,0.0,4.0,猛暑日
142,2001,滋賀県,60,54.0,8.0,46.0,NaN,NaN,NaN,真夏日
143,2001,滋賀県,60,NaN,NaN,NaN,3.0,0.0,3.0,猛暑日
236,2002,滋賀県,60,60.0,15.0,45.0,NaN,NaN,NaN,真夏日
237,2002,滋賀県,60,NaN,NaN,NaN,4.0,0.0,4.0,猛暑日
330,2003,滋賀県,60,36.0,23.0,13.0,NaN,NaN,NaN,真夏日
331,2003,滋賀県,60,NaN,NaN,NaN,0.0,0.0,0.0,猛暑日
424,2004,滋賀県,60,64.0,18.0,46.0,NaN,NaN,NaN,真夏日
425,2004,滋賀県,60,NaN,NaN,NaN,3.0,1.0,2.0,猛暑日


In [15]:
url = 'https://www.data.jma.go.jp/stats/data/mdrr/tem_rct/alltable/mxtemsadext00_rct.csv'
data = pd.read_csv(url, encoding='Shift-JIS')

In [16]:
data[data.地点.str.contains('東京')]

,観測所番号,都道府県,地点,国際地点番号,現在時刻(年),現在時刻(月),現在時刻(日),現在時刻(時),現在時刻(分),今日の最高気温(℃),...,昨日までの観測史上1位の値の品質情報,昨日までの観測史上1位の値を観測した起日（年）,昨日までの観測史上1位の値を観測した起日（月）,昨日までの観測史上1位の値を観測した起日（日）,昨日までの8月の1位の値,昨日までの8月の1位の値の品質情報,昨日までの8月の1位の値の起日（年）,昨日までの8月の1位の値の起日（月）,昨日までの8月の1位の値の起日（日）,統計開始年
382,44132,東京都,東京（トウキョウ）,47662.0,2023,8,17,13,0,34.5,...,8,2004,7,20,39.1,8,1994,8,3,1875
